In [212]:
# importing packege
import numpy as np
import math
import pandas as pd

# make sample
def MakeSample():
    dataSet = np.array([[0, 1, 1, 'yes'],
                        [0, 1, 0, 'no'],
                        [1, 0, 1, 'no'],
                        [1, 1, 1, 'no'],
                        [0, 1, 0, 'no'],
                        [0, 0, 1, 'no'],
                        [1, 0, 1, 'no'],
                        [1, 1, 0, 'no']])

    return dataSet

# calculate base entropy
def BaseEnt(dataSet):
    label = dataSet[:, -1]
    pos = []
    BaseEnt = []
    for i in pd.unique(label):
        pos.append(len(label[label == i]) / len(label))

#    for j in range(len(pos)):
#        temp = pos.pop()
#        BaseEnt.append(-temp*(math.log2(temp))-temp*(math.log2(temp)))

    BaseEnt = -pos[0]*math.log2(pos[0])-pos[1]*math.log2(pos[1])

    return label, pos, BaseEnt

dataSet = MakeSample()
label, pos, BaseEnt = BaseEnt(dataSet)


0.5435644431995964

In [179]:
label

[0.125, 0.875]

In [195]:
-pos[1]*math.log2(pos[1])-pos[0]*(math.log2(pos[0]))

0.5435644431995964

In [184]:
-pos[0]*math.log2(pos[0])

0.375

In [189]:
-pos[1]*(math.log2(pos[1]))

0.16856444319959643

In [175]:
(-(1/8))*(math.log(1/8))-((6/7)*(math.log(6/7)))

0.39205934684762944

In [176]:
(-(1/8))*(math.log2(1/8))-(7/8)*(math.log2(7/8))

0.5435644431995964

In [155]:
len(label)

7

In [115]:
pos.pop()

IndexError: pop from empty list

In [113]:
-pos.pop()#*math.log(pos.pop())

-0.8571428571428571

In [112]:
(-pos[0])*(math.log(pos[0]))

0.277987164150759

In [78]:
-BaseEnt[0] * math.log(BaseEnt[0])#-BaseEnt[1] * math.log(BaseEnt[1])

0.277987164150759

In [73]:
-BaseEnt[1] * math.log(BaseEnt[1])

0.13212915413765

In [49]:
pd.unique(label)

array(['yes', 'no'], dtype=object)

In [50]:
len(label[label == "yes"])

1